# Vehicle Routing Problem
[Murwan Siddig](mailto:msiddig@clemson.edu)

------------------------------------------------------------------------
## Two Index One Commodity Flow Formulation

In [1]:
include("data_VRP.jl");

In [2]:
#technical lines
using Random;
using Distributions;
using LinearAlgebra
using JuMP;
using Gurobi;
const GRB_ENV = Gurobi.Env();


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


In [3]:
#Function to extract the tour for each vehicle in operation 
function extractTour(n, sol)
    #Initilaize the an empty list for the lists of tours
    tours = [];
    #The number of vehicles in operation is equal to the number of arcs leaving the depot node 
    used_vehilces = sum(sol[1,j] for j=1:n);
    count = used_vehilces
    #Keep doing this loop until the tour for each vehicle in use is obtained 
    while count !=0
        count -=1
        tour = [1]  # Start at city 1 always
        cur_city = 1
        while true
            # Look for first arc out of current city
            for j = 1:n
                if sol[cur_city,j] >= 1-1e-6
                    # Found next city
                    push!(tour, j)
                    # Don't ever use this arc again
                    sol[cur_city, j] = 0.0
                    sol[j, cur_city] = 0.0
                    # Move to next city
                    cur_city = j
                    break
                end
            end
            # If we have come back to 1, stop
            if cur_city == 1
                break
            end
        end  # end while
        push!(tours, tour)
    end
    
    return tours
end

extractTour (generic function with 1 method)

In [4]:
#Two-index one-commodity flow formulation 
VRP2 = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
@variable(VRP2, x[1:nn,1:nn], Bin); 
@variable(VRP2, f[1:nn,1:nn]>=0);
@objective(VRP2, Min, sum(c[i,j]*x[i,j] for i=1:nn for j=1:nn));


for i = 1:nn
    @constraint(VRP2, x[i,i] == 0)
    @constraint(VRP2, f[i,i] == 0)
end                
                
@constraint(VRP2, sum(x[j,1] for j=2:nn)==sum(x[1,j] for j=2:nn));
@constraint(VRP2, sum(x[j,1] for j=2:nn) <=nv);
@constraint(VRP2, sum(x[1,j] for j=2:nn) <=nv);
for i=2:nn
    ind = [];
    for k=1:nn
        if k !=i
            push!(ind,k)
        end
    end
    @constraint(VRP2, sum(x[i,j] for j in ind)==sum(x[j,i] for j in ind));
    @constraint(VRP2, sum(x[i,j] for j in ind)==1);
    @constraint(VRP2, sum(x[j,i] for j in ind)==1);
end

for i=2:nn
    ind = [];
    for k=1:nn
        if k !=i
            push!(ind,k)
        end
    end
    @constraint(VRP2, sum(f[j,i] for j in ind)-sum(f[i,j] for j in ind)==q[i]);
end
for i=1:nn
    for j=1:nn
        @constraint(VRP2, f[i,j]<=(Q-q[i])*x[i,j]);
        @constraint(VRP2, q[j]*x[i,j]<=f[i,j]);
    end
end

In [5]:
@time optimize!(VRP2)
println("***********************************************")
println("***********************************************")
println(" solved! ")
println("===============================================")
tours = extractTour(nn, value.(x))
println("Tours = ",tours)
println("This solution is Feasible and Optimal")
println("Optimal objective value = ", objective_value(VRP2));

750.629110 seconds (39.05 M allocations: 1.953 GiB, 0.25% gc time)
***********************************************
***********************************************
 solved! 
Tours = Any[[1, 15, 29, 12, 5, 24, 4, 3, 7, 1], [1, 21, 6, 26, 11, 16, 23, 10, 9, 19, 30, 1], [1, 25, 28, 1], [1, 27, 8, 14, 18, 20, 32, 22, 1], [1, 31, 17, 2, 13, 1]]
This solution is Feasible and Optimal
Optimal objective value = 784.0
